### Imports

In [142]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf

import numpy as np
import pandas as pd
from sklearn import svm, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder

import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, Convolution2D
from keras.layers import MaxPooling2D, AveragePooling2D, MaxPool2D
from keras.layers.normalization import BatchNormalization

import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO)

### Functions

#### MNIST

In [144]:
"""
(1-split)% training data, split% test data
"""
def load_mnist(split = 0.10):
    train = pd.read_csv('train.csv')
    X = train.as_matrix()[:,1::] # All columns expect the first (label) column
    y = train.label.as_matrix() 
    
    # Create train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)
    
    return X_train, y_train, X_test, y_test

In [145]:
def kaggle_predict(model):
    # Predict kaggle test data
    test = pd.read_csv("test.csv").as_matrix().reshape(-1,28,28,1)
    pred = model.predict(kaggle_test)
    decoded = np.round(pred * np.asmatrix(range(10)).transpose()).astype(int)
    
    # Save as csv
    output = pd.DataFrame(decoded)
    output.index += 1
    output.index.name = 'ImageId'
    output.to_csv("predictions.csv", header=['Label'])

#### Noise

In [153]:
def create_model():
    
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax", name = "Output_Layer"))
    
    model.summary()
    
    return model

#### Add label noise

<b>Uniform label swapping</b> (sampling without replacement)

In [147]:
"""
For each n labels add alpha noisy labels
"""
def labelswap(y_train, alpha):
    lol = y_train.copy()
    n = int(y_train.shape[0]/(alpha+1))
    an = y_train.shape[0] - n
    print("{} clean labels, {} noisy labels, alpha={}".format(n,an,alpha))

    ind = np.random.choice(range(y_train.shape[0]), an, replace=False) # select an random labels
    shift = np.random.randint(1,9,an) # shift the label randomly by 1 to 9
    y_train[ind] = (y_train[ind] + shift) % 10 # calculate the new labels

    return y_train

#### Load dataset

In [148]:
X_train, y_train, X_test, y_test = load_mnist()

# Add label noise
alpha = 2
y_train = labelswap(y_train,alpha)

12600 clean labels, 25200 noisy labels, alpha=2


#### Pre-processing

In [149]:
# Normalize data to deal with covariate shift
scaling = MinMaxScaler(feature_range=(0,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

# Reshape to tensor (for CNN input in keras)
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

# One-hot encoding for softmax layer
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

C:\Users\dljva\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


#### Test model

In [154]:
model = create_model()
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
activation_3 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 32)        0         
__________

In [155]:
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data = (X_test,y_test))

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 11s - loss: 2.3102 - acc: 0.1478 - val_loss: 2.1758 - val_acc: 0.4129
Epoch 2/100
37800/37800 [==============================] - 10s - loss: 2.2537 - acc: 0.1878 - val_loss: 1.8642 - val_acc: 0.7924
Epoch 3/100
37800/37800 [==============================] - 10s - loss: 2.2329 - acc: 0.2044 - val_loss: 1.7558 - val_acc: 0.8162
Epoch 4/100
37800/37800 [==============================] - 10s - loss: 2.2138 - acc: 0.2190 - val_loss: 1.7464 - val_acc: 0.8693
Epoch 5/100
37800/37800 [==============================] - 10s - loss: 2.2015 - acc: 0.2213 - val_loss: 1.6668 - val_acc: 0.8824
Epoch 6/100
37800/37800 [==============================] - 10s - loss: 2.1845 - acc: 0.2369 - val_loss: 1.6422 - val_acc: 0.9343
Epoch 7/100
37800/37800 [==============================] - 10s - loss: 2.1773 - acc: 0.2414 - val_loss: 1.6014 - val_acc: 0.9176
Epoch 8/100
37800/37800 [=======================

#### Kaggle

In [156]:
test = pd.read_csv("test.csv").as_matrix().reshape(-1,28,28,1)
results = model.predict(test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Label')
submission = pd.concat([pd.Series(range(1,28001), name='ImageId'), results], axis=1)
submission.to_csv("predictions/labelnoise_adadelta_bn_b128.csv", index=False)

### Results
alpha = 2, 12600 clean 25200 noisy labels

<pre>
optimizer     epochs     batch size      train acc   val acc   kaggle acc
adadelta      100        128             0.4                   0.497
adam          100        128             0.6         0.60      0.38
adadelta      100        256             0.42        0.75      0.40
adadelta      100        64              0.38        0.82      0.47
adadelta                                             0.88      0.17
</pre>